# Machine Learning Structural Protein Sequences Classification for Drug Discovery

## 1. Introduction

Proteins are fundamental biomolecules that carry out essential functions within living organisms. Understanding the structure of a protein is critical for drug discovery, as the structural conformation often determines the protein’s function, interaction with other molecules, and its role in disease mechanisms. Traditional experimental methods to determine protein structure, such as X-ray crystallography and NMR spectroscopy, are time-consuming and expensive.

Advances in computational biology and machine learning offer an efficient alternative. By leveraging sequence-based features, deep learning models such as Long Short-Term Memory (LSTM) networks and transformer-based language models can predict protein structural classes directly from amino acid sequences. These approaches accelerate the identification of potential drug targets and facilitate the design of novel therapeutics.

### 1.1. Purpose

The purpose of this project is to develop a machine learning pipeline for classifying structural protein sequences into their respective structural classes, using sequence data alone. Specifically, this project aims to:

1. Preprocess protein sequences and convert them into machine-readable formats suitable for deep learning models.

2. Apply NLP techniques, including embeddings and sequence modeling, to capture meaningful patterns in protein sequences.

3. Compare the performance of different models, such as LSTM networks and large language models (LLMs), in structural protein classification.

4. Provide insights into how computational sequence analysis can accelerate drug discovery and facilitate the identification of novel therapeutic targets.

### 1.2. Dataset

The protein sequence data used is publicly available at [Kaggle](https://www.kaggle.com/code/davidhjek/protein-sequence-classification) in a raw and no-duplicates form. It was retrieved from Research Collaboratory for Structural Bioinformatics (RCSB) Protein Data Bank (PDB).

- `pdb_data_no_dups.csv` contains protein metadata which includes details on protein classification, extraction methods, etc.

| Column                     | Description                                                                                 |
| -------------------------- | ------------------------------------------------------------------------------------------- |
| `structureId`              | Unique identifier for each protein structure in the PDB (Protein Data Bank).                |
| `classification`           | Structural class or category of the protein (e.g., enzyme, transporter).                    |
| `experimentalTechnique`    | Method used to determine the protein structure (e.g., X-ray crystallography, NMR, cryo-EM). |
| `macromoleculeType`        | Type of macromolecule (e.g., protein, DNA, RNA).                                            |
| `residueCount`             | Number of amino acid residues in the protein chain.                                         |
| `resolution`               | Resolution of the protein structure (Ångströms), relevant for X-ray crystallography.        |
| `structureMolecularWeight` | Molecular weight of the protein structure (Daltons).                                        |
| `crystallizationMethod`    | Method used for crystallizing the protein (if applicable).                                  |
| `crystallizationTemp`      | Temperature used for protein crystallization (Kelvin or Celsius).                           |
| `densityMatthews`          | Matthews coefficient, a measure of crystal packing density.                                 |
| `densityPercentSol`        | Estimated solvent content (%) in the crystal.                                               |
| `pdbxDetails`              | Additional details about the structure (text description).                                  |
| `phValue`                  | pH at which the protein structure was determined.                                           |
| `publicationYear`          | Year the protein structure was published in the PDB.                                        |

    
- `pdb_data_seq.csv` contains >400,000 protein structure sequences.

| Column              | Description                                                                   |
| ------------------- | ----------------------------------------------------------------------------- |
| `structureId`       | Unique identifier for the protein structure (matches `pdb_data_no_dups.csv`). |
| `chainId`           | Identifier for the protein chain within the structure (A, B, C, etc.).        |
| `sequence`          | Amino acid sequence of the chain (one-letter codes).                          |
| `residueCount`      | Number of residues in this chain.                                             |
| `macromoleculeType` | Type of macromolecule (e.g., protein, DNA, RNA).                              |

## 2. Import Libraries and Datasets

### 2.1. Import Libraries

In [ ]:
# Core Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# NLP / Deep Learning (LSTM, embeddings)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Bidirectional
import tensorflow_hub as hub

# LLM / Transformers (protein language models)
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import sentencepiece

# Bioinformatics / Protein Utilities
from Bio import SeqIO

# Notebook Utilities
from tqdm.notebook import tqdm
import ipywidgets as widgets

c:\Users\palon\Envs\ml\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


### 2.2. Import Datasets

In [5]:
# Metadata
metadata = pd.read_csv("pdb_data_no_dups.csv")

# Sequences
sequences = pd.read_csv("pdb_data_seq.csv")

display(metadata.head())
display(sequences.head())

,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear
0,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,1.78,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0
1,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,NaN,NaN,2.00,38.45,NaN,NaN,1995.0
2,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,NaN,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
3,102D,DNA,X-RAY DIFFRACTION,DNA,24,2.20,7637.17,"VAPOR DIFFUSION, SITTING DROP",277.0,2.28,46.06,"pH 7.00, VAPOR DIFFUSION, SITTING DROP, temper...",7.0,1995.0
4,102L,HYDROLASE(O-GLYCOSYL),X-RAY DIFFRACTION,Protein,165,1.74,18926.61,NaN,NaN,2.75,55.28,NaN,NaN,1993.0


,structureId,chainId,sequence,residueCount,macromoleculeType
0,100D,A,CCGGCGCCGG,20,DNA/RNA Hybrid
1,100D,B,CCGGCGCCGG,20,DNA/RNA Hybrid
2,101D,A,CGCGAATTCGCG,24,DNA
3,101D,B,CGCGAATTCGCG,24,DNA
4,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,Protein


### 2.3. Merge Datasets

In [6]:
# Keep only needed columns from metadata
metadata_subset = metadata[['structureId', 'classification']]

# Merge sequences with classifications
data = sequences.merge(metadata_subset, on='structureId', how='inner')

# Drop missing labels if any
data = data[data['classification'].notnull()]

# Check
data.head()

,structureId,chainId,sequence,residueCount,macromoleculeType,classification
0,100D,A,CCGGCGCCGG,20,DNA/RNA Hybrid,DNA-RNA HYBRID
1,100D,B,CCGGCGCCGG,20,DNA/RNA Hybrid,DNA-RNA HYBRID
2,101D,A,CGCGAATTCGCG,24,DNA,DNA
3,101D,B,CGCGAATTCGCG,24,DNA,DNA
4,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,Protein,OXYGEN TRANSPORT


### 2.4. Analysis of Features

#### 2.4.1. Metadata

| Column                     | Use for modeling? | Notes                                                         |
| -------------------------- | ----------------- | ------------------------------------------------------------- |
| `structureId`              | ✅ Yes             | Key for merging sequences with labels                         |
| `classification`           | ✅ Yes             | Target variable (structural class)                            |
| `experimentalTechnique`    | ❌ No              | Optional metadata, not used for sequence-based ML             |
| `macromoleculeType`        | ❌ No              | Could filter out non-proteins, but not a feature for LSTM/LLM |
| `residueCount`             | ❌ No              | Sequence length is captured from sequences themselves         |
| `resolution`               | ❌ No              | Metadata, not used in current model                           |
| `structureMolecularWeight` | ❌ No              | Metadata                                                      |
| `crystallizationMethod`    | ❌ No              | Metadata                                                      |
| `crystallizationTempK`     | ❌ No              | Metadata                                                      |
| `densityMatthews`          | ❌ No              | Metadata                                                      |
| `densityPercentSol`        | ❌ No              | Metadata                                                      |
| `pdbxDetails`              | ❌ No              | Metadata                                                      |
| `phValue`                  | ❌ No              | Metadata                                                      |
| `publicationYear`          | ❌ No              | Metadata                                                      |

#### 2.4.2. Sequences

| Column              | Use for modeling? | Notes                                                                   |
| ------------------- | ----------------- | ----------------------------------------------------------------------- |
| `structureId`       | ✅ Yes             | Merge key                                                               |
| `chainId`           | ❌ Optional        | Could treat different chains separately; usually just keeps unique rows |
| `sequence`          | ✅ Yes             | **Main feature** for NLP/LSTM/LLM                                       |
| `residueCount`      | ❌ Optional        | Length can be derived from `sequence`                                   |
| `macromoleculeType` | ❌ Optional        | Could filter out non-proteins, usually redundant                        |
